# Import library

In [38]:
import torch
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import random
import torchvision
from typing import Any
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
from sklearn.metrics import accuracy_score



# Setup device and tensorboard

In [39]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
writer = SummaryWriter("logs_classification")
torch.manual_seed(42)

# Customized Dataset

In [40]:
class ImageDataset(Dataset):
    def __init__(self, list_root_dir: list, transform=None) -> None:
        super().__init__()
        self.list_root_dir = list_root_dir
        self.transform = transform
        
        self.image_paths =[]
        for root_dir in list_root_dir:
            self.image_paths.extend(glob.glob(os.path.join(root_dir,"with","*.jpg")))
            self.image_paths.extend(glob.glob(os.path.join(root_dir,"without","*.jpg")))
        random.shuffle(self.image_paths)
    def __len__(self) -> int:
        return len(self.image_paths)
    def __getitem__(self, index) -> Any:
        image_path = self.image_paths[index]
        
        label = 1 if image_path.split(os.sep)[-2] == "with" else 0
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
            
        return [image, label]

# Train one epoch

In [41]:
def train_model(model, loss_function, optimizer, train_loader, epoch=1):
    model.train()
    running_loss = 0.0
    total_samples = 0
    y_true = []
    y_pred = []
    for data in train_loader:     
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        # Clear gradient
        optimizer.zero_grad()
        # Calculate logits
        outputs = model(images)
        # Calculate loss
        loss = loss_function(outputs, labels)
        # Calculate gradient from loss
        loss.backward()
        # Update weight
        optimizer.step()

        # Calculate loss
        running_loss += loss.item() * images.size(0)
        # Calculate total sample in data_loader
        total_samples += images.size(0)
        # Calculte y_predict for evaluation
        predicted = torch.argmax(outputs, dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.detach().cpu().numpy())
    loss = running_loss/total_samples
    accuracy = accuracy_score(y_true, y_pred)
    writer.add_scalar("train/loss", loss, epoch)
    writer.add_scalar("train/accuracy", accuracy, epoch)
    return loss, accuracy

# Eval model



In [42]:
def eval_model(model, loss_function, test_dataloader, epoch):
    model.eval()
    total_samples = 0
    total_loss = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device)

            # Calculate logits
            outputs = model(images)
            
            # Calculate loss of outputs and y_true
            loss = loss_function(outputs, labels)
            total_loss += loss.item() * images.size(0)
            # Calculate total sample
            total_samples += images.size(0)
            
            # Calculte y_predict for evaluation
            predicted = torch.argmax(outputs, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.detach().cpu().numpy())
    loss = total_loss/total_samples
    accuracy = accuracy_score(y_true, y_pred)
    writer.add_scalar("test/loss", loss, epoch)
    writer.add_scalar("test/accuracy", accuracy, epoch)
    return loss, accuracy

# Model

In [43]:
model = torchvision.models.resnet34(num_classes=2)
model = model.to(device)
print(device)

cuda:0


# Optimizer


In [44]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Loss function

In [45]:
loss_function = torch.nn.CrossEntropyLoss()

# Train

In [46]:
list_root_dir = []
list_root_dir.append(os.path.join(os.getcwd(), "dataset", "gan_makeup_data_96"))
list_root_dir.append(os.path.join(os.getcwd(), "dataset", "mtdataset_96"))
transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop([96,96]),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
        torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])
epoch = 20
dataset = ImageDataset(list_root_dir, transform)

# Perform train-test split
train_dataset, test_dataset = random_split(dataset, [0.9,0.1])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

for i in range(50):
    train_loss, train_acc = train_model(model, loss_function, optimizer, train_loader, i)
    test_loss, test_acc = eval_model(model, loss_function, test_loader, i)
    print(f'''epoch {i}: train_loss {round(train_loss,4)}, train_acc {round(train_acc,4)}, test_loss {test_loss}, test_acc {round(test_acc,4)}''')
writer.flush()
writer.close()

epoch 0: train_loss 0.6742, train_acc 0.6654, test_loss 0.6192325431971304, test_acc 0.6273
epoch 1: train_loss 0.5583, train_acc 0.7101, test_loss 0.516021721672496, test_acc 0.7354
epoch 2: train_loss 0.5394, train_acc 0.7225, test_loss 0.5507777027798175, test_acc 0.7271
epoch 3: train_loss 0.5198, train_acc 0.7301, test_loss 0.5813605861735225, test_acc 0.6872
epoch 4: train_loss 0.5086, train_acc 0.7419, test_loss 0.4806480611521075, test_acc 0.7521
epoch 5: train_loss 0.4929, train_acc 0.7464, test_loss 0.46713274667743043, test_acc 0.7554
epoch 6: train_loss 0.4915, train_acc 0.7454, test_loss 0.5258640871071776, test_acc 0.7155
epoch 7: train_loss 0.4827, train_acc 0.7606, test_loss 0.5410944953337684, test_acc 0.7238
epoch 8: train_loss 0.4729, train_acc 0.7582, test_loss 0.4769845348031271, test_acc 0.7488
epoch 9: train_loss 0.4647, train_acc 0.7654, test_loss 0.4816878328902551, test_acc 0.772
epoch 10: train_loss 0.469, train_acc 0.7652, test_loss 0.44552787478870637, test

# Save model

In [ ]:
file_path = "classification.pth"
torch.save(model.state_dict(), file_path)

In [47]:
# print(len(dataset))

# fig, axes = plt.subplots(10, 10, figsize=(10,10))
# for i in range(10):
#     for j in range(10):
#         image = torchvision.transforms.ToPILImage()(dataset[i*10+j][0])
#         label = dataset[i*10+j][1]
#         axes[i][j].imshow(image)
#         axes[i][j].axis("off")
#         axes[i][j].set_title(label)
# plt.show()



In [48]:
# import numpy as np

# def cross_entropy_loss(logits, targets):
#     num_samples = targets.shape[0]

#     softmax_probs = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
#     a = np.arange(num_samples)
#     b = softmax_probs[np.arange(num_samples), targets]
#     log_probs = -np.log(softmax_probs[np.arange(num_samples), targets])

#     loss = np.sum(log_probs) / num_samples
#     return loss
# logits = np.array([[2,4],
#                   [5,2]], dtype=np.float32)
# target = np.array([0,1])

# loss = cross_entropy_loss(logits, target)
# print(loss)

# logits = torch.tensor([[2,4],
#                   [5,2]], dtype=torch.float32)
# target = torch.tensor([0,1])

# softmax = torch.nn.Softmax(dim=1)
# softmax_logits = softmax(logits)
# print(softmax_logits)
# loss_function = torch.nn.CrossEntropyLoss()
# loss = loss_function(logits, target)
# print(loss)